In [ ]:
#load data
import numpy as np
celeba = np.load('drive/My Drive/celeba.npy')

In [ ]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import load_model
from matplotlib import pyplot

In [ ]:
# definition of the discriminator
def discriminator(input_shape=(64,64,3)):
  model = Sequential()
  
  
  model.add(Conv2D(64, (5,5), padding='same', input_shape=input_shape))
  
  model.add(LeakyReLU(alpha=0.2))
  

  # downsample to 32x32
  model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
  
  model.add(LeakyReLU(alpha=0.2))
 

  # downsample to 16x16
  model.add(Conv2D(128, (5,5), strides=(2,2),padding='same'))
  
  model.add(LeakyReLU(alpha=0.2))
 
  # downsample to 8x8
  model.add(Conv2D(256, (5,5), strides=(2,2), padding='same'))
  
  model.add(LeakyReLU(alpha=0.2))
  

  # downsample to 4x4
  model.add(Conv2D(512, (5,5), strides=(2,2), padding='same'))
  
  model.add(LeakyReLU(alpha=0.2))
  
  
  # classifier
  model.add(Flatten())

  model.add(Dropout(0.4))

  model.add(Dense(1, activation='sigmoid'))
  
  optim = Adam(lr=0.0002, beta_1=0.5)
  
  #compile model
  model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
  
  return model

In [ ]:
#definition of the generator
def generator(latent_dimension):
    model = Sequential()
    
    
    n_nodes = 512 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dimension))
    
    model.add(LeakyReLU(alpha=0.2))
   
    model.add(Reshape((4, 4, 512)))
    # upsample to 8X8
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 64x64
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2),padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # output layer 64x64x3
    model.add(Conv2D(3, (5,5), activation='tanh',padding='same'))
    return model

In [ ]:
def define_gan(gen, dis):
  
  dis.trainable = False
  
  model = Sequential()
  
  #add generator
  model.add(gen)
  
  #add the discriminator
  model.add(dis)
  
  #compile model
  optim = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=optim)
  
  return model

In [ ]:
#function used to generate batch of real images 
def generate_real_samples(dataset, n_batch):
  
  #n_batch random images from the dataset
  ix = randint(0, dataset.shape[0], n_batch)
  
  X = dataset[ix]
  
  #map images into the range [-1,1]
  X = (X.astype('float32')-127.5)/127.5
  
  y = ones((n_batch, 1))
  return X, y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dimension, n_batch):
  
  # generate points in the latent space
  lat_input = randn(latent_dimension * n_batch)
  
  # reshape into a batch of inputs for the network
  lat_input = lat_input.reshape(n_batch, latent_dimension)
  
  return lat_input

In [ ]:
# generate a batch of fake samples
def generate_fake_samples(gen, latent_dimension, n_batch):
  
  # generate points in latent space
  lat_input = generate_latent_points(latent_dimension, n_batch)
  # predict outputs
  X = gen.predict(lat_input)
  
  # fake class labels
  y = zeros((n_batch, 1)) 
  return X, y


In [ ]:
def train(gen, dis, gan, dataset,latent_dimension, n_epochs=50, n_batch=64):
  
  half_batch = int(n_batch / 2)
  
  #number of batches per epoch
  bat_per_epo = int((dataset.shape[0] / n_batch))
  
  #list instantiated to store the losses
  dloss_real = []
  dloss_fake = []
  gloss = []
  
  
  
  
  for i in range(n_epochs):
    
    
    for j in range(bat_per_epo):
      
      #generate a batch of real samples
      X_real, y_real = generate_real_samples(dataset, half_batch)

      #update discriminator model weights
      d_loss1, _ = dis.train_on_batch(X_real, y_real)
        
      #generate a batch of fake generated samples
      X_fake, y_fake = generate_fake_samples(gen, latent_dimension, half_batch)
        
      #update discriminator model weights
      d_loss2, _ = dis.train_on_batch(X_fake, y_fake)

      #prepare points in latent space as input for the generator
      X_gan = generate_latent_points(latent_dimension, n_batch)
      
      #create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      
      #update the generator 
      g_loss = gan.train_on_batch(X_gan, y_gan)
      
      #store the losses
      dloss_real.append(d_loss1)
      dloss_fake.append(d_loss2)
      gloss.append(g_loss)

      #print the epoch, batch number annd losses
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
      (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))


      
    
    #at the end of each epoch, save the models and the losses
    losses = [np.array(dloss_real),np.array(dloss_fake),np.array(gloss)]
    
    save_training(i,gan, gen, dis, dataset, latent_dimension,losses)
    
  

In [ ]:
#save a plot of n*n images
def save_plot(examples, epoch, n=4):
  
  # scale from [-1,1] to [0,1]
  examples = (examples + 1) / 2.0
  
  # plot images
  for i in range(n * n):
  
  # define subplot
    pyplot.subplot(n, n, 1 + i)
  # turn off axis
    pyplot.axis('off')
  # plot raw pixel data
    pyplot.imshow(examples[i])
  
  # save plot to file
  filename = 'drive/My Drive/model/generated_plot_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()



#evaluate the discriminator, generate and save a plot of fake images, save models
def save_training(epoch,gan, gen, dis, dataset, latent_dimension,losses, n_samples=16):
  
  # prepare real samples
  X_real, y_real = generate_real_samples(dataset, n_samples)

  # evaluate discriminator on real examples
  _, acc_real = dis.evaluate(X_real, y_real, verbose=0)
  
  # prepare fake examples
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  
  # evaluate discriminator on fake examples
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  
  # summarize discriminator performance
  print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
  
  # save plot
  save_plot(x_fake, epoch)
  
  #save models
  filename = 'drive/My Drive/model/generator_model_%03d.h5' % (epoch + 42)
  gen.save(filename)
  filename = 'drive/My Drive/model/gan_model_%03d.h5' % (1)
  gan.save(filename)
  filename = 'drive/My Drive/model/d_model_%03d.h5' % (1)
  dis.trainable = True
  
  for layer in d_model.layers:
    layer.trainable = True
  d_model.save(filename)
  d_model.trainable = False
  for layer in d_model.layers:
    layer.trainable = False
  
  np.savez('drive/My Drive/Losses/losses.npz', *losses)
  
  



In [ ]:
#training

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = discriminator()
# create the generator
g_model = generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model,celeba[:192599], latent_dim)

In [ ]:

#restart training from saved models


def load_models():
    
    filename = 'drive/My Drive/model/d_model_001.h5'
    dis = load_model(filename, compile=True)
    dis.trainable = True
    
    for layer in d_model.layers:
        layer.trainable = True
   
    
    filename = 'drive/My Drive/model/generator_model_041.h5'
    gen = load_model(filename, compile=True)
    
    gan = define_gan(gen, dis)
    gan.summary()
    return dis, gen, gan



In [ ]:

# create a plot of n*n generated images
latent_dim = 100
def plot_generated(examples, n):
  # plot images
  for i in range(n * n):
    # define subplot
    pyplot.subplot(n, n, 1 + i)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(examples[i, :, :])
    pyplot.show()
# load model
model = load_model('drive/My Drive/model/generator_model_040.h5')
# generate images
latent_points = generate_latent_points(latent_dim, 25)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
plot_generated(X, 5)